# Вступительное задание на кафедру компании ABBYY
## Андрей Сонин

### Условие

Представим, что вы &mdash; разработчик в отделе машинного обучения и перед вами поставлена задача разработать модуль автоматического обнаружения русскоязычных текстов. На вход подаётся большой набор документов, среди которых нужно обнаружить тексты, содержащие значительные фрагменты на русском языке. [Пример входных данных](https://drive.google.com/file/d/1kVY4C-taT1h8XBvrUyHCJoLWKmXYWKzU/view?usp=sharing)
 
В компании нет обучающих данных, но вы можете собрать необходимые данные самостоятельно, разрешается использовать любые данные для обучения, готовые модели и программные средства, например, https://www.sketchengine.eu/guide/create-a-corpus-from-the-web.
 
Требуется реализовать функцию predict, принимающую путь к папке, в которой содержатся тексты, сохраненные в формате .txt (utf-8). В каждом файле содержится ровно один текст. Функция генерирует в заданной папке файл с предсказаниями prediction.csv в формате (filename;answer), где answer &mdash; 1 (содержит русский язык) или 0 (не содержит русского языка).
 
Также реализуйте функцию predict_once, которая принимает строку с текстом и выводит ответ в стандартный вывод.
 
Обратите внимание, что задача не является контестом, в качестве решения нужно предоставить код на Python (можно Jupyter) или C++. Другой язык программирования разрешается использовать только по предварительной договоренности. Решение должно содержать хотя бы минимальные тесты, оценку качества и анализ ошибок.

## Системные требования
- Python: >= 3.7
- ОС: `Linux/macOS`

## Предварительные приготовления
### Загрузка дополнительных библиотек

In [1]:
from importlib.util import find_spec
from os import system


def install_package(name: str,
                    *,
                    conda_suffix: str = '',
                    pip_suffix: str = '') -> None:
    """Checks if a module is downloaded.
    If not, tries to install it first with 'conda', then, if unsuccessful, with 'pip'.

    Args:
        name:          package name
        conda_suffix:  additional part of 'conda' installation command
        pip_suffix:    additional part of 'pip' installation command

    Returns:
        None
    """
    if (
        not find_spec(name)
        and system(f'conda install -y {name} {conda_suffix}')
        and system(f'pip install {name} {pip_suffix}')
    ):
        raise RuntimeError(f"Cannot install '{name}' module")


install_package('nltk')
install_package('langdetect', conda_suffix='-c conda-forge')
install_package('requests')

### Загрузка примера входных данных из Google Drive

In [2]:
# >>> Google drive downloading script >>>
from os import PathLike
from os.path import isdir
from typing import Union
from requests import Session, Response


def download_file_from_google_drive(google_id: str,
                                    destination: Union[PathLike, str]) -> None:
    URL = 'https://docs.google.com/uc?export=download'

    session = Session()

    response = session.get(URL, params={'id': google_id}, stream=True)
    token = get_confirm_token(response)

    if token:
        response = session.get(
            URL,
            params={'id': google_id, 'confirm': token},
            stream=True
        )

    save_response_content(response, destination)    


def get_confirm_token(response: Response) -> None:
    for key, value in response.cookies.items():
        if key.startswith('download_warning'):
            return value
    return None


def save_response_content(response: Response,
                          destination: Union[PathLike, str],
                          *,
                          chunk_size: int = 100_000_000) -> None:
    with open(destination, 'wb') as f:
        for chunk in response.iter_content(chunk_size):
            if chunk:  # filters out keep-alive new chunks
                f.write(chunk)


if not isdir('data'):
    download_file_from_google_drive('1kVY4C-taT1h8XBvrUyHCJoLWKmXYWKzU', 'data.zip')
    !unzip -o data.zip > /dev/null && rm data.zip

### Подключение всех необходимых библиотек

In [3]:
from glob import iglob
from itertools import compress
from collections import Counter
from pickle import dump, load
from operator import itemgetter
from csv import reader as csv_reader
from multiprocessing import Pool, cpu_count
from os import PathLike, makedirs
from os.path import getsize, basename, isfile, join as join_path
from pathlib import Path
from typing import Iterable, Iterator, Sequence, Callable, TypeVar

import nltk
from langdetect import detect
from langdetect.lang_detect_exception import LangDetectException
from nltk import sent_tokenize, WordPunctTokenizer


nltk.download('punkt')

T_co = TypeVar('T_co', covariant=True)

[nltk_data] Downloading package punkt to /home/asonin/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


## Основной код
- Чтобы иметь дополнительные гарантии в корректности питоновского кода, были расставлены аннотации типов, а код проверен с помощью статического анализатора `MyPy`.

- Для идентификации языка текста был использован наивный байесовский классификатор [`langdetect`](https://github.com/Mimino666/langdetect), обученный на 55 языках, включая русский.

- Решение о наличии значительных фрагментов русского языка в тексте основывается на предположении о том, что каждое конкретное предложение в тексте написано на определённом языке. Для каждого предложения `langdetect` определяет его язык, и если доля предложений на русском превышает фиксированный порог, то считается, что данный текст содержит значительные фрагменты русского текста.

- Для уменьшения времени детектирования функция `predict` выполнена параллельной.

In [4]:
# >>> Auxiliary utils >>>

def split(iterable: Iterable[T_co],
          n: int,
          *,
          collector: Callable[[Iterable[T_co]], Sequence[T_co]] = tuple) -> Iterator[Sequence[T_co]]:
    """Splits an iterable into 'n' equal pieces, if possible.
    If not, the first few pieces will be 1 longer than the last ones.

    >>> tuple(split(range(10), 3))
    ((0, 1, 2, 3), (4, 5, 6), (7, 8, 9))

    >>> list(split(range(7), 4))
    [(0, 1), (2, 3), (4, 5), (6,)]

    >>> list(split(range(7), 4, collector=list))
    [[0, 1], [2, 3], [4, 5], [6]]

    >>> list(split(range(3), 5, collector=list))
    [[0], [1], [2], [], []]

    Args:
        iterable:   iterable to split
        n:          number of pieces
        collector:  split holder
    Returns:
        split iterator
    """

    container = collector(iterable)
    split_size, remainder = divmod(len(container), n)
    n_elements = [split_size] * n
    for i in range(remainder):
        n_elements[i] += 1

    first = last = 0
    for n in n_elements:
        last += n
        yield container[first:last]
        first = last


class MutableStr:
    __slots__ = ('data',)

    def __init__(self):
        self.data = ''

In [5]:
# >>> Main functions >>>

def _is_russian_sentence(sentence: str) -> bool:
    """Checks if the given sentence is in Russian.

    Args:
        sentence:  sentence of interest

    Returns:
        Result of checking
    """
    try:
        return detect(sentence) == 'ru'
    except LangDetectException:
        return False


def _is_russian_text(text: str, thresh: float) -> bool:
    """Checks if the given text is in Russian.

    Comes to this conclusion if the length of Russian sentences
    is at least 'thresh' of the length of the entire text.

    Args:
        text:    text of interest
        thresh:  identification threshold

    Returns:
        Result of checking
    """
    sentences = sent_tokenize(text, language='russian')

    total_sent_len = total_ru_sent_len = 0
    for sent in sentences:
        sent_len = sum(not ch.isspace() for ch in sent)
        total_sent_len += sent_len
        if _is_russian_sentence(sent):
            total_ru_sent_len += sent_len

    try:
        return (total_ru_sent_len / total_sent_len) >= thresh
    except ZeroDivisionError:
        return False


def _predict_for_files(file_names: Iterable[Union[PathLike, str]], thresh: float) -> str:
    """Takes file names and for each file determines whether it contains text in Russian.

    Comes to the conclusion that the file contains Russian text
    if the length of Russian sentences is at least 'thresh' of the length of the entire text.

    Args:
        file_names:  paths to files of interest
        thresh:      identification threshold. Should be from [0, 1]

    Returns:
        CSV string containing entries of 'filename;answer\n' format. Answer can be 0 or 1
    """
    answers = ''

    for file_name in file_names:
        with open(file_name, 'r') as file:
            content = file.read()
        ru = _is_russian_text(content, thresh)
        answers += f"{basename(file_name)};{'1' if ru else '0'}\n"

    return answers


def predict(path: Union[PathLike, str],
            *,
            n_jobs: int = -1,
            thresh: float = 0.1) -> None:
    """Takes a path to the folder and for each file
    with the .txt extension in it determines whether it contains text in Russian.

    Comes to the conclusion that the file contains Russian text
    if the length of Russian sentences is at least 'thresh' of the length of the entire text.

    Args:
        path:    path to the folder of interest
        n_jobs:  number of jobs to run in parallel. Must be positive or -1 (use all CPUs)
        thresh:  identification threshold. Should be from [0, 1]

    Returns:
        None
    """
    if not 0 <= thresh <= 1:
        raise ValueError("Parameter 'thresh' should be from [0, 1]")
    if n_jobs == -1:
        n_jobs = cpu_count()

    answers = MutableStr()
    def callback(one_proc_answers: str) -> None:
        answers.data += one_proc_answers

    with Pool(n_jobs) as task_pool:
        file_name_iter = iglob(join_path(path, '*.txt'))
        for file_name_chunk in split(file_name_iter, n_jobs):
            if not file_name_chunk:
                break
            task_pool.apply_async(
                _predict_for_files,
                (file_name_chunk, thresh),
                callback=callback
            )
        task_pool.close()
        task_pool.join()

    with open(join_path(path, 'prediction.csv'), 'w') as ans_file:
        ans_file.write(answers.data)


def predict_once(text: str, *, thresh: float = 0.1) -> None:
    """Takes a string and determines whether it contains Russian text,
    printing the result to the standard output (1 - yes, 0 - no).

    Comes to this conclusion if the length of Russian sentences
    is at least 'thresh' of the length of the entire text.

    Args:
        text:    text of iterest
        thresh:  identification threshold. Should be from [0, 1]

    Returns:
        None
    """
    if not 0 <= thresh <= 1:
        raise ValueError("Parameter 'thresh' should be from [0, 1]")
    print('1' if _is_russian_text(text, thresh) else '0')

In [6]:
%%time
DATA_DIR = Path('data')

predict(DATA_DIR)

CPU times: user 126 ms, sys: 246 ms, total: 373 ms
Wall time: 12.7 s


In [7]:
def test_predict_once():
    print('Result')

    predict_once(
    """En la sede del Consejo Federal de Inversiones se realizó el lanzamiento de la octava edición de la Noche de las Casas de Provincias, que tendrá lugar el próximo jueves 28 de 17 a 21 en todas las representaciones oficiales en Buenos Aires.
    El secretario Bernardo Abruzzese participó del evento, que es organizado cada año por el Consejo Federal de Representaciones Oficiales (Confedro), organismo que nuclea a las casas de provincia, y que cuenta con el apoyo de diferentes organismos nacionales y gubernamentales, entre los que se destaca el Consejo Federal de Inversiones (CFI).
    Este año, nuestra provincia contará con una variada propuesta cultural y turística, en la que se destaca la realización de la Marcha de los Bombos por el microcentro porteño, con la participación del “Indio” Froilán González y Tere Castronuovo, un homenaje a Vitillo Ábalos, y la presencia del Dúo Coplanacu como artista central.
    Bajo la consigna “Recorré tu país en una noche”, todos los años las representaciones oficiales de las provincias en Buenos Aires abren sus puertas para promocionar y difundir cada una de las provincias desde lo turístico, cultural, su producción, etc.
    Propuestas gastronómicas, degustación de productos, shows tecnológicos, artesanías, danza, muestras artísticas, sorteos, juegos, actividades para los más chicos, música en vivo y stands de productos regionales reciben a las miles de personas que se acercan a las Casas a conocer y disfrutar una fantástica noche federal para toda la familia.
    Cuenta con el apoyo del Ministerio del Interior, el Gobierno de la Ciudad de Buenos Aires, la Cámara Argentina de Turismo y la Federación Argentina de Asociaciones de Empresas de Viajes y Turismo."""
    )

    predict_once(
    """Министерство культуры планирует ужесточить правила охраны музеев, использовав опыт аэропортов. Об этом пишет «Интерфакс» со ссылкой на директора музейного департамента министерства Владислава Кононова.
    Кононов заявил, что Минкультуры планирует внедрить на входе в музеи «систему выявления неадекватных посетителей». По его словам, музеи находятся на «передовой линии общественного внимания», и в связи с этим в них бывают «случаи девиантного поведения отдельных лиц».
    Директор музейного департамента сослался на опыт аэропортов, говоря о необходимости противостоять непредсказуемым людям в музеях. В аэропортах, отметил он, наблюдение незаметно, но если человек привлечет внимание, к нему подойдут, проверят документы и спросят о самочувствии. То же самое необходимо внедрять в музеях, сказал Кононов. «Шедевры требуют к себе особого отношения в плане безопасности», — отметил он.
    В январе 2019 года из Третьяковской галереи украли картину Архипа Куинджи «Ай-Петри. Крым» из коллекции Русского музея. До этого мужчина повредил в Третьяковской галерее картину Репина «Иван Грозный и сын его Иван». Ущерб оценили в 20 миллионов рублей."""
    )

    predict_once(
        'Саркофаг украшен фигурными рельефами, а также рельефами, несущими функцию декоративного орнамента'
    )

    predict_once(
        'Арифметиканың барлыҡҡа килеүенең сәбәбе булып иҫәпләүҙәргә һәм ауыл хужалығын үҙәкләштергәндәге мәсьәләләр менән бәйле иҫәп-хисапҡа практик мохтажлыҡ тора.Фән хәл итеүҙе талап иткән мәсьәләләр менән бергә үҫешә. Арифметиканың үҫешенә грек математиктары ҙур өлөш индерә — атап әйткәндә, философтар-һандар ярҙамында донъяның бөтә законлыҡтарын аңларға һәм тасуирларға тырышыусы пифагорсылар.'
    )

    print('\nGround truth', '0', '1', '1', '0', sep='\n')

test_predict_once()

Result
0
1
1
1

Ground truth
0
1
1
0


Видим, что для распространённых языков всё работает хорошо.

Но наш классификатор ошибочно определяет башкирский язык как русский, что неудивительно для наивного байесовского классификатора, анализирующего символьные n-граммы, ведь башкирский язык также использует кириллицу, но в обучении не встречался.

Давайте поможем нашему классификатору отличать русский язык от других языков, использующих кириллицу. Для этого выделим наиболее распространённые слова русского языка.

## Загрузка и обработка корпуса русской википедии

In [8]:
DUMP_DIR = Path('dumps')
RU_WORD_FREQ_DUMP = DUMP_DIR / 'ru_word_freqs.pkl'

if not isfile(RU_WORD_FREQ_DUMP):
    # Загрузка предварительно сохранённого дампа результата обработки корпуса (if-ом ниже)
    makedirs(DUMP_DIR, exist_ok=True)

    RU_WORD_FREQ_DUMP_BZ2 = f'{RU_WORD_FREQ_DUMP}.bz2'
    if not isfile(RU_WORD_FREQ_DUMP_BZ2):
        download_file_from_google_drive('100vQH23cMAVa7Ii3ngQ4zRHySY7LWY47', RU_WORD_FREQ_DUMP_BZ2)
    !pbzip2 -dk $RU_WORD_FREQ_DUMP_BZ2 2> /dev/null || bzip2 -dk $RU_WORD_FREQ_DUMP_BZ2
    del RU_WORD_FREQ_DUMP_BZ2


if getsize(RU_WORD_FREQ_DUMP) < 1_000_000:
    # Уберите это условие, если хотите повторить обработку корпуса
    # !!! REQUIRES MORE THAN 50GB

    !wget https://zenodo.org/record/3827903/files/wikipedia-ru-2018.txt.gz -O dumps/wikipedia-ru-2018.txt.gz
    !gzip -d dumps/wikipedia-ru-2018.txt.gz

    with open(DUMP_DIR / 'wikipedia-ru-2018.txt', 'r') as wiki_dump_file:
        wiki_dump = wiki_dump_file.read()
    wiki_dump = wiki_dump.lower()

    ru_word_freqs = Counter(
        tok for tok in WordPunctTokenizer().tokenize(wiki_dump)
        if 'А' <= tok[0] <= 'я'  # Getting rid of non-word and non-Russian tokens
    )
    del wiki_dump
    with open(RU_WORD_FREQ_DUMP, 'wb') as pkl:
        dump(ru_word_freqs, pkl)
else:
    with open(RU_WORD_FREQ_DUMP, 'rb') as pkl:
        ru_word_freqs = load(pkl)

Сохраним 200_000 наиболее распространённых слов во множество и модифицируем функцию `_is_russian_sentence`.

In [9]:
ru_most_common = frozenset(map(itemgetter(0), ru_word_freqs.most_common(200_000)))


def _is_russian_sentence(sentence: str, *, tokenizer=WordPunctTokenizer()) -> bool:
    """Checks if the given sentence is in Russian.

    Args:
        sentence:   sentence of interest
        tokenizer:  word tokenizer

    Returns:
        Result of checking
    """
    try:
        res = detect(sentence)
    except LangDetectException:
        return False
    if res == 'ru':
        words = tuple(
            tok for tok in tokenizer.tokenize(sentence.lower())
            if tok[0].isidentifier()  # Getting rid of non-word tokens
        )
        n_rus_words = sum(
            1 for word in words
            if word in ru_most_common  # or all('А' <= ch <= 'я' for ch in word)
        )
        try:
            return (n_rus_words / len(words)) >= 0.8  # Hard coded constant!
        except ZeroDivisionError:
            pass
    return False

In [10]:
test_predict_once()

Result
0
1
1
0

Ground truth
0
1
1
0


Замечательно. Башкирский язык теперь определяется правильно.

Давайте теперь вручную проаннотируем некоторое подможество текстов из предоставленной Вами выборки. Это необходимо для более надёжной оценки качества модели.

In [11]:
# Manual annotation result
is_russian_truth = {
    '1000.txt': False,
    '1001.txt': False,
    '1002.txt': False,
    '1003.txt': False,
    '1004.txt': False,
    '1005.txt': False,
    '1006.txt': False,
    '1007.txt': False,
    '1008.txt': False,
    '1009.txt': False,
    '1010.txt': False,
    '1011.txt': False,
    '1012.txt': False,
    '1013.txt': False,
    '1014.txt': False,
    '1015.txt': True,
    '1016.txt': False,
    '1017.txt': False,
    '1018.txt': False,
    '1019.txt': False,
    '1021.txt': False,
    '1022.txt': False,
    '1023.txt': True,
    '1024.txt': False,
    '1025.txt': False,
    '1026.txt': True,
    '1027.txt': False,
    '1028.txt': False,
    '1029.txt': False,
    '1030.txt': False,
    '1031.txt': False,
    '1032.txt': False,
    '1033.txt': False,
    '1034.txt': False,
    '1035.txt': False,
    '1036.txt': False,
    '1037.txt': False,
    '1038.txt': True,
    '1039.txt': False,
    '1040.txt': False,
    '1041.txt': False,
    '1042.txt': False,
    '1043.txt': False,
    '1045.txt': False,
    '1046.txt': False,
    '1047.txt': False,
    '1048.txt': False,
    '1049.txt': False,
    '1050.txt': False,
    '1051.txt': False,
    '1052.txt': False,
    '1054.txt': False,
    '1055.txt': False,
    '1056.txt': False,
    '1057.txt': False,
    '1058.txt': False,
    '1060.txt': False,
    '1061.txt': True,
    '1062.txt': False,
    '1063.txt': False,
    '1064.txt': False,
    '1065.txt': False,
    '1066.txt': False,
    '1067.txt': False,
    '1068.txt': False,
    '1069.txt': False,
    '1070.txt': False,
    '1071.txt': False,
    '1072.txt': False,
    '1073.txt': False,
    '1074.txt': False,
    '1075.txt': False,
    '1076.txt': False,
    '1077.txt': False,
    '1078.txt': False,
    '1079.txt': False,
    '1080.txt': False,
    '1081.txt': False,
    '1082.txt': False,
    '1083.txt': False,
    '1084.txt': False,
    '1085.txt': False,
    '1086.txt': False,
    '1087.txt': False,
    '1088.txt': False,
    '1089.txt': True,
    '1090.txt': False,
    '1091.txt': True,
    '1092.txt': False,
    '1093.txt': False,
    '1094.txt': True,
    '1095.txt': False,
    '1096.txt': False,
    '1097.txt': True,
    '1098.txt': False,
    '1099.txt': False
}

Вызовем функцию `predict` ещё раз и выведем в стандартный поток те имена файлов из подвыборки, класс которых был предсказан неправильно.

In [12]:
predict(DATA_DIR)

with open(DATA_DIR / 'prediction.csv', 'r') as file:
    is_russian_pred = {
        file: ru == '1'
        for file, ru in csv_reader(file, delimiter=';')
    }

for file_basename, ru_ground_truth in is_russian_truth.items():
    if is_russian_pred[file_basename] != ru_ground_truth:
        print(file_basename, ru_ground_truth)

Таковых не нашлось, что может объясняться малым объёмом тестовой выборки.

Как бы то ни было, можно видеть, что данный классификатор поставленную задачу выполняет успешно.